In [1]:
pip install openai azure-storage-blob azure-identity


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/80.6 kB ? eta -:--:--
     ---------------------------------------- 80.6/80.6 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/408.6 kB ? eta -:--:--
   --------------------------------------- 408.6/408.6 kB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/187.6 kB ? eta -:--:--
   --------------------------------------- 187.6/187.6 kB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.9 kB ? eta -:--:--
   --------------------------------------- 198.9/198.9 kB 11.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/113.2 kB ? eta -:--:--
   ---------------------------------------- 113.2/113.2 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
# openai.api_type = "azure_ad"
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_key = token.token
# openai.api_version = "2022-12-01"


api_key = os.environ.get("AZURE_OPENAI_API_KEY")

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')


Test if completitions work:

In [3]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT"), 
  api_key=api_key,
  api_version="2024-02-01"
)


deployment_name='gpt-4o-mini' #This will correspond to the custom name you chose for your deployment when you deployed a model. 
    
response = client.chat.completions.create(
  model=deployment_name,
  messages=[
    {"role": "system", "content": "YOU ARE USEFUL ASSISTANT"},
    {"role": "user", "content": "How are you"}
  ]
)

print(response.choices[0].message.content)


I'm just a computer program, but I'm here and ready to help you! How can I assist you today?


Do it in a streaming fashion:

Test if embeddings work:

In [6]:
response = client.embeddings.create(
    input = "Your text string goes here",
    model= "text-embedding-ada-002"  # model = "deployment_name".
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        -0.007578954566270113,
        -0.0055061643943190575,
        0.011402026750147343,
        -0.0247525442391634,
        -0.024873483926057816,
        0.039802949875593185,
        -0.010347154922783375,
        -0.009480412118136883,
        -0.013384111225605011,
        -0.00991042423993349,
        -0.011717816814780235,
        0.008217253722250462,
        -0.014338199980556965,
        0.0077603659592568874,
        0.010044802911579609,
        -0.0049820877611637115,
        0.022777177393436432,
        -0.0017082883277907968,
        0.015319163911044598,
        -0.010279965586960316,
        0.0049182577058672905,
        0.012208298780024052,
        0.004834271036088467,
        0.010568879544734955,
        -0.006574474740773439,
        -0.00044596908264793456,
        0.005707732401788235,
        -0.012853316031396389,
        0.01634044200181961,
        0.004454651847481728,
        0.0063762660138309,
        -0.0

Test if tokenizer works:

In [7]:
tokens = encoding.encode("Hello world!")
print(tokens)
print(len(tokens))

[9906, 1917, 0]
3


# Ejercicio 1: Generación de texto con diferentes prompts
# Objetivo: Experimentar con distintos prompts para observar cómo varía la respuesta del modelo.


In [8]:
prompts = [
    "Write a poem about the sea",
    "Summarize the latest advancements in AI",
    "Explain quantum physics in simple terms",
    "Generate a list of creative startup ideas"
]

for prompt in prompts:
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    print(f"Prompt: {prompt}")
    print(f"Response: {response.choices[0].message.content}\n")

Prompt: Write a poem about the sea
Response: Upon the edge where sky meets tide,  
The whispers dance, in rhythm they glide,  
Beneath the sun's warm golden gleam,  
The sea unfurls its endless dream.  

With waves like laughter, bright and free,  
They crash and curl, a symphony,  
Each crest a story, each trough a sigh,  
In the embrace of the swell, the world drifts by.  

Azure depths hold secrets old,  
In coral gardens, wonders bold,  
The silver fish in fleeting play,  
Spin tales of sunlight, kelp, and spray.  

As twilight falls, the horizon bleeds,  
Into hues of crimson, where the heart concedes,  
The moon ascends, a watchful eye,  
Casting silver nets on the world's sigh.  

The briny air, a lover’s breath,  
Speaks of mysteries, life, and death,  
For in the sea, both wild and vast,  
Lies the echo of the future and shadows of the past.  

So here I stand, with heart in hand,  
Awed by the beauty of this boundless land,  
With every tide that ebbs and flows,  
The sea tea

# Ejercicio 2: Comparación de similitud entre textos usando embeddings
# Objetivo: Calcular la similitud coseno entre dos fragmentos de texto.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity


In [10]:
text1 = "Machine learning is fascinating."
text2 = "Artificial intelligence drives machine learning."
text3 = "Cooking is an essential skill."

embedding1 = client.embeddings.create(input=text1, model=EMBEDDING_MODEL).data[0].embedding
embedding2 = client.embeddings.create(input=text2, model=EMBEDDING_MODEL).data[0].embedding
embedding3 = client.embeddings.create(input=text3, model=EMBEDDING_MODEL).data[0].embedding


import numpy as np  
from sklearn.metrics.pairwise import cosine_similarity  

embedding1 = np.array(embedding1).reshape(1, -1)  
embedding2 = np.array(embedding2).reshape(1, -1)  
embedding3 = np.array(embedding3).reshape(1, -1)  

similarity_1_2 = cosine_similarity(embedding1, embedding2)[0][0]  
similarity_1_3 = cosine_similarity(embedding1, embedding3)[0][0]  

print(f"Similarity between text1 and text2: {similarity_1_2:.2f}")  
print(f"Similarity between text1 and text3: {similarity_1_3:.2f}")  

Similarity between text1 and text2: 0.88
Similarity between text1 and text3: 0.78


# Ejercicio 3: Tokenización de frases complejas
# Objetivo: Practicar la tokenización de frases más largas y analizar la cantidad de tokens generados.

In [11]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Exploring the universe through the lens of scientific discovery is an unending journey of knowledge.",
    "AI is transforming industries at an unprecedented pace."
]

for sentence in sentences:
    tokens = encoding.encode(sentence)
    print(f"Sentence: {sentence}")
    print(f"Tokens: {tokens}")
    print(f"Number of tokens: {len(tokens)}\n")

Sentence: The quick brown fox jumps over the lazy dog.
Tokens: [791, 4062, 14198, 39935, 35308, 927, 279, 16053, 5679, 13]
Number of tokens: 10

Sentence: Exploring the universe through the lens of scientific discovery is an unending journey of knowledge.
Tokens: [45053, 5620, 279, 15861, 1555, 279, 18848, 315, 12624, 18841, 374, 459, 653, 2518, 11879, 315, 6677, 13]
Number of tokens: 18

Sentence: AI is transforming industries at an unprecedented pace.
Tokens: [15836, 374, 46890, 19647, 520, 459, 31069, 18338, 13]
Number of tokens: 9



# Ejercicio 4: Generación de texto en streaming
# Objetivo: Implementar una generación de texto que muestre la respuesta en tiempo real (streaming).

In [12]:
import time

stream_response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a storytelling assistant."},
        {"role": "user", "content": "Tell me a story about a brave knight."}
    ],
    stream=True
)

for chunk in stream_response:
    if chunk.choices:
        print(chunk.choices[0].delta.content or "", end="", flush=True)
        time.sleep(0.05)

print("\n\n--- End of Story ---")

Once upon a time, in a land where valleys kissed the sky and rivers sang as they flowed, there lived a brave knight named Sir Cedric. Sir Cedric was known throughout the kingdom of Eldoria for his valor and unwavering sense of justice. Clad in gleaming armor that shone like the sun, he rode a magnificent steed named Valor, whose strength was as legendary as the knight's own.

The kingdom was a peaceful place, governed by the wise King Alden, until a dark shadow fell upon it. A fearsome dragon, known as Malakar, descended from the Misty Mountains, unleashing terror upon the villages. The creature had an insatiable hunger for gold, and wherever it roamed, it laid waste to crops and homes. The people trembled in fear, whispering of the dragon's fiery breath and its scales as hard as steel.

King Alden called forth his bravest knights, but many were hesitant, having seen the devastation wrought by Malakar. It was Sir Cedric who stepped forward, resolute and unyielding. “I shall face the dr

## Ejercicio 2: Comparación de similitudes semánticas entre dos párrafos

En este ejercicio, aprenderás a usar los embeddings de OpenAI para calcular la similitud semántica entre dos párrafos, lo que te permitirá comparar la cercanía entre textos utilizando la métrica de similitud coseno.

Pasos:

Usa el modelo de embeddings text-embedding-ada-002 de OpenAI.
Calcular la similitud coseno entre dos fragmentos de texto.
Imprimir el valor de similitud entre los párrafos.

Resultado esperado:

Similitud entre text1 y text2: 0.94
Similitud entre text1 y text3: 0.15
Explicación: Los estudiantes aprenderán cómo usar embeddings para evaluar la similitud semántica entre diferentes fragmentos de texto. En este caso, text1 y text2 son similares en contenido relacionado con la inteligencia artificial, por lo que la similitud es alta (0.94), mientras que text3 habla sobre actividades al aire libre, lo que genera una similitud baja con los otros textos.



In [52]:
# Frases de ejemplo
text1 = "Artificial intelligence is transforming the world in every sector."
text2 = "AI is revolutionizing industries globally, changing the way we work."
text3 = "I love hiking and spending time in nature."

embedding1 = client.embeddings.create(input=text1, model=EMBEDDING_MODEL).data[0].embedding
embedding2 = client.embeddings.create(input=text2, model=EMBEDDING_MODEL).data[0].embedding
embedding3 = client.embeddings.create(input=text3, model=EMBEDDING_MODEL).data[0].embedding


# Calcular la similitud coseno
similarity_1_2 = cosine_similarity([embedding1], [embedding2])[0][0]
similarity_1_3 = cosine_similarity([embedding1], [embedding3])[0][0]

print(f"Similitud entre text1 y text2: {similarity_1_2:.2f}")
print(f"Similitud entre text1 y text3: {similarity_1_3:.2f}")


Similitud entre text1 y text2: 0.93
Similitud entre text1 y text3: 0.73


## Ejercicio 3: Extracción de palabras clave de un texto utilizando embeddings

Este ejercicio tiene como objetivo enseñar cómo usar OpenAI para extraer palabras clave relevantes de un texto, basándose en la similitud de los embeddings.

Pasos:

Usa un fragmento de texto.
Genera embeddings para cada palabra.
Evalúa la similitud de cada palabra con el texto completo para identificar las palabras más relevantes.

Resultado esperado:

Palabra: artificial - Similitud: 0.92
Palabra: intelligence - Similitud: 0.88
Palabra: healthcare - Similitud: 0.83
Palabra: finance - Similitud: 0.80
Palabra: education - Similitud: 0.78
Palabra: revolutionize - Similitud: 0.70

Explicación: El modelo de OpenAI se utiliza para evaluar la similitud de cada palabra clave con el contexto general del texto. Las palabras más relevantes, como "artificial" e "intelligence", tienen una mayor similitud con el texto completo, mientras que otras, como "revolutionize", tienen una menor similitud.


In [61]:


# Fragmento de texto
text = "Artificial intelligence has the potential to revolutionize multiple sectors including healthcare, finance, and education."

# Generar embeddings para el texto completo
text_embedding = client.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding

# Listado de palabras clave a evaluar
keywords = ["artificial", "intelligence", "healthcare", "revolutionize", "finance", "education"]

# Calcular embeddings para cada palabra y compararlos con el texto
word_similarities = {}
for word in keywords:
    word_embedding = client.embeddings.create(input=word, model=EMBEDDING_MODEL).data[0].embedding
    similarity = cosine_similarity([text_embedding], [word_embedding])[0][0]
    word_similarities[word] = similarity

# Mostrar palabras clave ordenadas por relevancia
sorted_keywords = sorted(word_similarities.items(), key=lambda x: x[1], reverse=True)

for word, similarity in sorted_keywords:
    print(f"Palabra: {word} - Similitud: {similarity:.2f}")


Palabra: healthcare - Similitud: 0.83
Palabra: intelligence - Similitud: 0.81
Palabra: education - Similitud: 0.80
Palabra: revolutionize - Similitud: 0.79
Palabra: finance - Similitud: 0.79
Palabra: artificial - Similitud: 0.79


## Ejercicio 4: Generación de resúmenes con OpenAI
Enunciado:
En este ejercicio, el objetivo es generar un resumen breve de un texto largo utilizando el modelo de OpenAI.

Pasos:

Usa un texto largo y genera un resumen de este utilizando un prompt adecuado.
Configura correctamente la solicitud a OpenAI para obtener un resumen conciso.

Resultado esperado:

Resumen generado:
Artificial intelligence (AI) is the intelligence demonstrated by machines, contrasting with natural intelligence. AI is defined as the study of intelligent agents, which are devices that perceive their environment and take actions to maximize their goals. The field of AI aims to mimic human cognitive functions, such as learning and problem-solving, though tasks considered to require intelligence are often excluded from AI as machines improve.

Explicación: Este ejercicio enseña cómo utilizar OpenAI para generar resúmenes de textos largos. El modelo utiliza el contexto dado para condensar la información sin perder el sentido de lo que se quiere transmitir.


In [74]:


# Texto largo
long_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals. Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem-solving". As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect. AI research has been defined as the study of "intelligent agents"."

"""

prompt = f"Provide a brief summary of the following text:\n{long_text}"
# Solicitar resumen

response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant."},
            {"role": "user", "content": prompt}
        ],
    max_tokens=100
    )

# Mostrar resumen
print(f"Resumen generado:\n{response.choices[0].message.content}")


Resumen generado:
Artificial intelligence (AI) refers to the intelligence exhibited by machines, differing from the natural intelligence of humans and animals. It is defined in leading AI textbooks as the study of "intelligent agents" — devices that perceive their environment and act to achieve goals. Commonly, AI describes machines that replicate cognitive functions associated with human intelligence, like learning and problem-solving. As machines advance, tasks once deemed to require intelligence are often excluded from the definition of AI, a trend referred to as the AI effect.


## Ejercicio 5: Generación de respuestas basadas en conocimiento específico
Enunciado:
Este ejercicio te ayudará a generar respuestas detalladas y útiles a partir de un contexto o conocimiento específico, por ejemplo, sobre la historia de la informática.

Pasos:

Crea un conjunto de datos que contenga información básica sobre un tema.
Genera una respuesta a una pregunta específica utilizando ese contexto como referencia.

Resultado esperado:

Charles Babbage is known as the father of computing.

Explicación: Este ejercicio demuestra cómo los modelos de OpenAI pueden generar respuestas útiles basadas en información específica proporcionada en un contexto. Los estudiantes aprenderán cómo usar esta capacidad para obtener respuestas precisas dentro de un marco de conocimiento.


In [80]:

# Contexto sobre la historia de la informática
context = """
The history of computer science dates back to the early 19th century, with key figures such as Charles Babbage, who is considered the "father of computing" for his invention of the Analytical Engine. In the mid-20th century, computers became more practical, with the development of the first programmable computers and the invention of the transistor. Later, the development of the internet revolutionized the field and led to the rise of modern computing technologies.
"""

# Pregunta
question = "Who is known as the father of computing?"

# Solicitar respuesta basada en el contexto

response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant."},
            {"role": "user", "content": f"Based on the following information, answer the question:\n{context}\n\nQuestion: {question}\nAnswer:"}
        ],
    max_tokens=100
    )

# Mostrar respuesta
print(f"Respuesta generada:\n{response.choices[0].message.content}")


Respuesta generada:
Charles Babbage is known as the father of computing.



## Ejercicio 6: Análisis de emociones en un texto

En este ejercicio, el objetivo es enseñar cómo realizar un análisis básico de emociones en un texto utilizando los modelos de OpenAI. El modelo debería identificar la emoción principal expresada en un texto.

Pasos:

Escribe varios textos que expresen diferentes emociones.
Usa OpenAI para clasificar la emoción principal de cada texto.

Resultado esperado:

Texto: I am so happy! Today was the best day of my life.
Emoción: Happiness

Texto: I feel really sad and upset after hearing the news.
Emoción: Sadness

Texto: The project is going great, I'm feeling confident and motivated.
Emoción: Confidence

Explicación: Los estudiantes aprenderán a utilizar OpenAI para realizar análisis emocionales de textos y comprender cómo los modelos pueden identificar emociones basadas en el contenido proporcionado.


In [84]:

# Textos de ejemplo con emociones
texts = [
    "I am so happy! Today was the best day of my life.",
    "I feel really sad and upset after hearing the news.",
    "The project is going great, I'm feeling confident and motivated.",
]

# Solicitar análisis de emociones
for text in texts:
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant that analise sentiment."},
            {"role": "user", "content": f"Analyze the emotion in the following text:\n{text}\nEmotion:"}
        ],
    max_tokens=10
    )
    print(f"Texto: {text}\nEmoción: {response.choices[0].message.content.strip()}\n")


Texto: I am so happy! Today was the best day of my life.
Emoción: The emotion in the text is overwhelmingly positive. The

Texto: I feel really sad and upset after hearing the news.
Emoción: The emotion expressed in the text is primarily sadness and

Texto: The project is going great, I'm feeling confident and motivated.
Emoción: The emotion in the text is positive. It expresses

